<a href="https://colab.research.google.com/github/Rishitha110506/IBS-02/blob/main/IBS2%20lab%201.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install biopython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 32.8 MB/s eta 0:00:00


In [3]:
!pip install biopython scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 51.0 MB/s eta 0:00:00


In [6]:
from google.colab import files
uploaded = files.upload()


Saving alzheimers_dataset.fasta to alzheimers_dataset.fasta


In [7]:
import os
os.listdir()


['.config', 'alzheimers_dataset.fasta', 'sample_data']

In [8]:
from Bio import SeqIO, pairwise2
import numpy as np
from collections import Counter
from sklearn.metrics.pairwise import cosine_similarity

# =========================
# 1️⃣ Load FASTA file
# =========================

fasta_file = "alzheimers_dataset.fasta"

sequences = []
ids = []

for record in SeqIO.parse(fasta_file, "fasta"):
    sequences.append(str(record.seq))
    ids.append(record.id)

print("Total sequences BEFORE redundancy removal:", len(sequences))

# =========================
# 2️⃣ Convert sequences to k-mer vectors
# =========================

def kmer_vector(seq, k=3):
    kmers = [seq[i:i+k] for i in range(len(seq)-k+1)]
    counts = Counter(kmers)
    return counts

all_kmers = set()
kmer_counts = []

for seq in sequences:
    counts = kmer_vector(seq)
    kmer_counts.append(counts)
    all_kmers.update(counts.keys())

all_kmers = sorted(list(all_kmers))

matrix = []

for counts in kmer_counts:
    vector = [counts.get(kmer, 0) for kmer in all_kmers]
    matrix.append(vector)

matrix = np.array(matrix)

# =========================
# 3️⃣ Cosine similarity redundancy removal
# =========================

threshold = 0.90
keep_indices = []

for i in range(len(matrix)):
    keep = True
    for j in keep_indices:
        sim = cosine_similarity([matrix[i]], [matrix[j]])[0][0]
        if sim >= threshold:
            keep = False
            break
    if keep:
        keep_indices.append(i)

unique_sequences = [sequences[i] for i in keep_indices]

print("Total sequences AFTER cosine similarity filtering:", len(unique_sequences))

# =========================
# 4️⃣ Global alignment scores (first 20)
# =========================

print("\nGlobal Alignment Scores (first 20 pairs):")

scores = []

for i in range(min(20, len(unique_sequences)-1)):
    score = pairwise2.align.globalxx(
        unique_sequences[i],
        unique_sequences[i+1],
        score_only=True
    )
    scores.append(score)
    print(f"Seq{i} vs Seq{i+1} Score:", score)

# =========================
# 5️⃣ Average alignment score
# =========================

average_score = np.mean(scores)
print("\nAverage Global Alignment Score:", average_score)

print("\nPipeline completed successfully ✅")


Total sequences BEFORE redundancy removal: 1229
Total sequences AFTER cosine similarity filtering: 496

Global Alignment Scores (first 20 pairs):
Seq0 vs Seq1 Score: 304.0
Seq1 vs Seq2 Score: 25.0
Seq2 vs Seq3 Score: 31.0
Seq3 vs Seq4 Score: 17.0
Seq4 vs Seq5 Score: 20.0
Seq5 vs Seq6 Score: 13.0
Seq6 vs Seq7 Score: 17.0
Seq7 vs Seq8 Score: 63.0
Seq8 vs Seq9 Score: 38.0
Seq9 vs Seq10 Score: 9.0
Seq10 vs Seq11 Score: 29.0
Seq11 vs Seq12 Score: 42.0
Seq12 vs Seq13 Score: 28.0
Seq13 vs Seq14 Score: 6.0
Seq14 vs Seq15 Score: 7.0
Seq15 vs Seq16 Score: 23.0
Seq16 vs Seq17 Score: 65.0
Seq17 vs Seq18 Score: 180.0
Seq18 vs Seq19 Score: 128.0
Seq19 vs Seq20 Score: 263.0

Average Global Alignment Score: 65.4

Pipeline completed successfully ✅
